In [1]:
import pandas as pd

file1 = "supplier_data1.xlsx"
file2 = "supplier_data2.xlsx"

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)


In [2]:
# Count nulls in all columns
null_counts = df1.isnull().sum()
print("Null counts:")
print(null_counts)

# Get value counts for each column
print("\nValue counts:")
for column in df1.columns:
    print(f"\nColumn: {column}")
    print(df1[column].value_counts(dropna=False))

Null counts:
Quality/Choice       0
Grade                0
Finish               0
Thickness (mm)       0
Width (mm)           0
Description          0
Gross weight (kg)    0
RP02                 0
RM                   0
Quantity             0
AG                   0
AI                   0
dtype: int64

Value counts:

Column: Quality/Choice
Quality/Choice
3rd    19
2nd    18
1st    13
Name: count, dtype: int64

Column: Grade
Grade
C100S    26
C300S    14
C200S    10
Name: count, dtype: int64

Column: Finish
Finish
gebeizt und geglüht    19
gebeizt                17
ungebeizt              14
Name: count, dtype: int64

Column: Thickness (mm)
Thickness (mm)
2.75    4
2.48    3
2.32    2
2.88    2
2.51    2
2.08    2
2.74    2
2.43    2
2.18    1
2.21    1
2.89    1
2.53    1
2.77    1
2.83    1
2.80    1
2.01    1
2.30    1
2.12    1
2.37    1
2.72    1
2.82    1
2.14    1
2.22    1
2.31    1
2.20    1
2.86    1
2.69    1
2.98    1
2.03    1
2.54    1
2.55    1
2.84    1
2.52    1
2.46    1

In [3]:
# Count nulls in all columns
null_counts = df2.isnull().sum()
print("Null counts:")
print(null_counts)

# Get value counts for each column
print("\nValue counts:")
for column in df2.columns:
    print(f"\nColumn: {column}")
    print(df2[column].value_counts(dropna=False))

Null counts:
Material       0
Description    0
Article ID     0
Weight (kg)    0
Quantity       0
Reserved       0
dtype: int64

Value counts:

Column: Material
Material
S235JR          19
HDC             13
DX51D +AZ150    10
DX51D +Z140      8
Name: count, dtype: int64

Column: Description
Description
Material is not Oiled    18
Material is Oiled        17
Material is Painted      15
Name: count, dtype: int64

Column: Article ID
Article ID
23048203    1
23045724    1
23048028    1
23045099    1
23046741    1
23047239    1
23049907    1
23044447    1
23047489    1
23040273    1
23040905    1
23043967    1
23049672    1
23040547    1
23049617    1
23043836    1
23049333    1
23041897    1
23040756    1
23041086    1
23047543    1
23045292    1
23047408    1
23043348    1
23049106    1
23044610    1
23049627    1
23041902    1
23046057    1
23041966    1
23043884    1
23048868    1
23046459    1
23043788    1
23044935    1
23049059    1
23047860    1
23047939    1
23047348    1
23045961

### After some basic EDA above, it is evident that there are no null values. However, the columns of the 2 datasets do not overlap completely. Therefore, the new columns in the combined dataset will have NaN or null values, since it is best not to impute with unreasonable assumptions, and also keep some distinction between itmes from the supplier1 dataset and the supplier2 dataset (in addition to the 'source' flag)

### We now proceed by first re-naming the columns in both datasets. The name are all in lower-case and consist only of letters and under-scores. The 'Material' column from supplier2 dataset is renamed to 'grade', since its values seem to follow the pattern of the 'grade' column in supplier1 dataset.

### Also, the previous EDA showed that the 'finish' and the 'description' column in supplier1 dataset have german language phrases whereas the 'description' column in supplier2 dataset has information in the English language. Hence, the 'finish' and the 'description' column of supplier 1 dataset are translated to their English translations.

### We also add a flag named 'source' that allows to distinguish between rows from the supplier1 dataset and the supplier2 dataset in the combined inventory dataset.

In [4]:
df1_clean = df1.rename(columns={
    "Quality/Choice": "quality",
    "Grade": "grade",
    "Finish": "finish",
    "Thickness (mm)": "thickness_mm",
    "Width (mm)": "width_mm",
    "Description": "description",
    "Gross weight (kg)": "weight_kg",
    "Quantity": "quantity",
})

finish_map = {
    "gebeizt und geglüht": "pickled and annealed",
    "gebeizt": "pickled",
    "ungebeizt": "unpickled"
}

description_map = {
    "Längs- oder Querisse": "longitudinal or transverse cracks",
    "Sollmasse (Gewicht) unterschritten": "nominal weight undershot",
    "Kantenfehler - FS-Kantenrisse": "edge defect - FS edge cracks"
}

df1_clean["finish"] = df1_clean["finish"].replace(finish_map)
df1_clean["description"] = df1_clean["description"].replace(description_map)
df1_clean["source"] = "supplier1"


df2_clean = df2.rename(columns={
    "Material": "grade",
    "Description": "description",
    "Article ID": "article_id",
    "Weight (kg)": "weight_kg",
    "Quantity": "quantity",
    "Reserved": "reserved"
})

# Add missing columns to align with supplier1 schema
for col in ["quality", "finish", "thickness_mm", "width_mm", "RP02", "RM", "AG", "AI"]:
    df2_clean[col] = pd.NA

df2_clean["source"] = "supplier2"


common_cols = [
    "quality", "grade", "finish", "thickness_mm", "width_mm",
    "description", "weight_kg", "quantity", "RP02", "RM", "AG", "AI",
    "article_id", "reserved", "source"
]

df1_final = df1_clean.reindex(columns=common_cols)
df2_final = df2_clean.reindex(columns=common_cols)

# Concatenate into unified dataset
inventory_dataset = pd.concat([df1_final, df2_final], ignore_index=True)


output_path = "inventory_dataset.csv"
inventory_dataset.to_csv(output_path, index=False)

# output_path


/var/folders/4z/jqjm7y1n685fk_nxyyd4rf1h0000gn/T/ipykernel_16123/1720065157.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  inventory_dataset = pd.concat([df1_final, df2_final], ignore_index=True)


### Viewing the first few rows of the combined inventory dataset and also a basic insight into the number of null values and the value counts of the combined dataset.

In [5]:
inventory_dataset.head()

,quality,grade,finish,thickness_mm,width_mm,description,weight_kg,quantity,RP02,RM,AG,AI,article_id,reserved,source
0,3rd,C200S,pickled and annealed,2.77,1100,longitudinal or transverse cracks,13983,0.00,333.6,606.2,16.11,0.0054,NaN,NaN,supplier1
1,3rd,C300S,unpickled,2.65,1075,longitudinal or transverse cracks,13047,0.00,717.7,0.0,16.11,0.0046,NaN,NaN,supplier1
2,3rd,C100S,pickled and annealed,2.20,1100,edge defect - FS edge cracks,14155,10.84,368.9,0.0,0.00,0.0061,NaN,NaN,supplier1
3,2nd,C100S,pickled,2.86,1100,longitudinal or transverse cracks,11381,22.87,368.9,601.7,0.00,0.0062,NaN,NaN,supplier1
4,1st,C300S,unpickled,2.88,1050,nominal weight undershot,10072,22.87,0.0,1213.0,0.00,0.0041,NaN,NaN,supplier1


In [6]:
# Count nulls in all columns
null_counts = inventory_dataset.isnull().sum()
print("Null counts:")
print(null_counts)

# Get value counts for each column
print("\nValue counts:")
for column in inventory_dataset.columns:
    print(f"\nColumn: {column}")
    print(inventory_dataset[column].value_counts(dropna=False))

Null counts:
quality         50
grade            0
finish          50
thickness_mm    50
width_mm        50
description      0
weight_kg        0
quantity         0
RP02            50
RM              50
AG              50
AI              50
article_id      50
reserved        50
source           0
dtype: int64

Value counts:

Column: quality
quality
NaN    50
3rd    19
2nd    18
1st    13
Name: count, dtype: int64

Column: grade
grade
C100S           26
S235JR          19
C300S           14
HDC             13
C200S           10
DX51D +AZ150    10
DX51D +Z140      8
Name: count, dtype: int64

Column: finish
finish
NaN                     50
pickled and annealed    19
pickled                 17
unpickled               14
Name: count, dtype: int64

Column: thickness_mm
thickness_mm
NaN     50
2.75     4
2.48     3
2.43     2
2.08     2
2.74     2
2.32     2
2.51     2
2.88     2
2.12     1
2.30     1
2.01     1
2.80     1
2.83     1
2.37     1
2.18     1
2.53     1
2.89     1
2.72     1
2.